# Create album

In [23]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling1D
from sklearn.preprocessing import StandardScaler
from keras.models import Model
from keras.layers import *    
import os 
import pyreadr
import numpy as np
import pyreadr
import pandas as pd
import butterfly.album
import butterfly.CNN
from itertools import combinations 
import multiprocessing
from joblib import Parallel, delayed

In [24]:
#Import the data
DF = pyreadr.read_r('/Users/mxenoc/Desktop/workspace/butterfly/data/omics.RData')
#DF = pyreadr.read_r('/home/mxenoc/workspace/butterfly/data/omics.RData')
DF = DF["DF"]

In [25]:
# Choose your predicted omics, size for the image and number of features we are predicting
response = 'immune'
pixels = 40
features = 62

response = [col for col in DF if col.startswith(response)]
response.append("patientID")
response.append("trimester")    
response_df = DF[response]
#Make sure your B trimesters are properly converted to 4

In [26]:
#Get every combination of patient IDs in pairs of m's
# Function which returns subset or r length from n 
m = 2
IDs = response_df['patientID'].unique()
IDs = IDs.tolist()
exclude = list(combinations(IDs, m))

In [27]:
predictors = ['rna', 'plasma_l', 'microb', 'immune', 'metabol', 'plasma_s']

In [28]:
albums = Parallel(n_jobs=6, prefer = "threads")(delayed(butterfly.album.create_album)
                                                (DF, predictors[al], pixels) 
                                                for al in range(6))

In [ ]:
yy = response_df.drop(['patientID', 'trimester'], axis =1 ).values

In [29]:
prediction, observed = zip(*Parallel(n_jobs=3)(delayed(butterfly.CNN.model_multi)
                                                               (response_df, exclude, cv, albums, pixels, features) 
                                                               for cv in range(len(exclude))))

ValueError: Error when checking target: expected activation_3 to have shape (62,) but got array with shape (534,)

In [ ]:
pred = pd.concat(prediction)
obs  = pd.concat(observed)

In [ ]:
correl = pred.corrwith(obs, axis = 0) 
correl[correl < 0] = 0
np.mean(correl)